In [10]:
# Author: Joey Li, Atsushi Hu, Becca Shapiro
# Last Updated: 7/20/2018

# This runs a benchmarking procedure on a given IBM Q backend for whatever circuits you specify.
# To calculate success rate, specify your desired outputs. The desired outputs can be obtained
# by first running on the local_qasm_simulator to get an idea of what the outputs should be.

# You may repurpose this script as needed. 

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, QISKitError
from qiskit import available_backends, register, execute, get_backend
from qiskit.tools.visualization import circuit_drawer, plot_histogram, plot_state
from qiskit.tools.qi.qi import state_fidelity
import random, pickle, time, datetime, xlsxwriter
import numpy as np
import scipy.io as spio
import importlib
import circuit_builder
importlib.reload(circuit_builder)
import encoding_schemes
importlib.reload(encoding_schemes)

#setup
import sys, getpass
try:
    sys.path.append("../../") # go to parent dir
    import Qconfig
    qx_config = {
        "APItoken": Qconfig.APItoken,
        "url": Qconfig.config['url']}
    print('Qconfig loaded from %s.' % Qconfig.__file__)
    print()
except:
    APItoken = getpass.getpass('Please input your token and hit enter: ')
    qx_config = {
        "APItoken": APItoken,
        "url":"https://quantumexperience.ng.bluemix.net/api"}
    print('Qconfig.py not found in qiskit-tutorial directory; Qconfig loaded using user input.')
    
#matlab stuff
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from numpy import matrix
from math import pi

#set 
QX_URL = "https://quantumexperience.ng.bluemix.net/api"
QX_TOKEN = "06704f99fc6dd68ab5ae2663f0861723406de8b4553cacdabdf500292c937717442daa90c2ffa19109cfb9114adb7c87711a08860d34d1da9332ac664e4e24a7"

#register token and url
register(QX_TOKEN, QX_URL)

# query for available backends
print("Available backends: ", available_backends())

backend = "ibmqx4"

# This function deals with the case when circuits contains a length one circuit, which is parsed
# as a string rather than an array. If we find a string we just return it
def hasString(circuits):
    strings = []
    newArray = []
    for element in circuits:
        if isinstance(element,np.ndarray):
            newArray.append(element)
            continue
        else:
            strings.append(element)
    if not strings:
        return None
    strings = np.asanyarray(strings)
    return [strings[0]]
    
        

# This function is not robust. Ties between circuits of equal length are broken arbitrarily.
# Sometimes, it may matter, because two implementations that are the same length may involve
# different numbers of gates and may have different performances. However, from current data,
# it seems that the shortest circuit is usually unique, so this is not currently a concern.
def getShortest(circuits):
    circuit = hasString(circuits)
    if circuit is None:
        circuit = min(circuits, key = len)
    return [circuit]

# Code for extracting gate constructions
extract = spio.loadmat('shortcengates.mat', squeeze_me = True)
H1 = extract['H1cen']

# IMPORTANT SETTINGS
# Update the scheme to reflect your encoding scheme
scheme = encoding_schemes.FourTwoTwo()

# Update desired outputs to reflect your circuit
desiredOutputs = [["0000","1111"], ["0101","1010"], ["1100","0011"],["0110","1001"]]
#desiredOutputs = [["00"], ["10"], ["01"], ["11"]]

# Initial compilers for 00, 01, 10, 11, currently only works for two qubits
initialStates = [1,2,3]

# How many reps you want each circuit (including state preparation) to run
reps = 9

# Specify circuits you want to run
# If you specify your circuits by hand, insert an array of circuits, where each circuit is array of strings.
# Also note that the circuit builder assumes you number qubits 1 to n, and will output your results in that
# order as well.
circuits = H1

# Keep this if you only want the shortest circuit
circuits = getShortest(circuits)
circuits = np.append(circuits[0], circuits[0])
circuits = [circuits]
print(circuits)



# These update automatically with the scheme
n = scheme.n
validOutputs = scheme.validOutputs

startTime = time.time()

# Main benchmarking procedure
for circuit in circuits:
    
    for compiler in initialStates:
        print("Circuit is: "+str(circuit))
        print("Compiler is: "+str(compiler))
            
        avgSuccess = 0

        for trial in range(reps):
            
            # Standard setup
            qr = QuantumRegister(n)
            cr = ClassicalRegister(n)
            qc = QuantumCircuit(qr,cr)
            
            # Sets the input
            scheme.prepareZeros(qc)
            
            # Maps the input info into actual physical qubits. Qubits 1 to n are displayed left to right in that order
            perm = scheme.perm
            scheme.setPerm()
            
            # Sets our initial state to 00, 01, 10, or 11 as appropriate
            scheme.setInput(qc, compiler)
            
            # Build the circuit
            registerPerm = circuit_builder.buildCirc(qc, circuit, perm)
            
            # Register perm makes sure measurements go to right registers
            for i in range(n):
                qc.measure(qr[registerPerm[i]],cr[i])
            
            # Standard execution code
            job = execute(qc, backend)
            result = job.result()
            data = result.get_counts(qc)
            
            # Prints raw data
            print(data)
            
            # Clean and process data
            successes = 0
            totalValid = 0
            for key, value in data.items():
                if key in desiredOutputs[compiler]:
                    successes += value
                if key in validOutputs:
                    totalValid += value
            
            # If we hit this, something went very wrong with our specification of desiredOutputs
            if totalValid == 0:
                print("ERROR WITH VALID INPUTS")
                
            else:
                # print("Success rate is: "+ str(successes/totalValid))
                avgSuccess += successes/totalValid
        
        # Calculate and display information
        avgSuccess /= reps
        print("Average of success rates is: "+ str(avgSuccess)+ "\n")

endTime = time.time()
print("Time running: "+str(endTime-startTime))

Qconfig loaded from C:\Users\ddrsq\OneDrive\IBM Q\qecc_benchmark\Qconfig.py.

Available backends:  ['ibmqx2', 'local_unitary_simulator', 'ibmq_qasm_simulator', 'ibmqx5', 'local_statevector_simulator', 'ibmqx4', 'local_qasm_simulator', 'local_clifford_simulator']
[array(['Permute 2  1  4  3', 'CNOT 4  2', 'CNOT 2  4', 'CNOT 1  2',
       'CNOT 1  4', 'H 1  2  3  4', 'CZ 1  4', 'CZ 2  4', 'H 1  2  3',
       'Permute 1  4  2  3', 'CNOT 1  2', 'Permute 2  1  4  3',
       'CNOT 4  2', 'CNOT 2  4', 'CNOT 1  2', 'CNOT 1  4', 'H 1  2  3  4',
       'CZ 1  4', 'CZ 2  4', 'H 1  2  3', 'Permute 1  4  2  3',
       'CNOT 1  2'], dtype=object)]
Circuit is: ['Permute 2  1  4  3' 'CNOT 4  2' 'CNOT 2  4' 'CNOT 1  2' 'CNOT 1  4'
 'H 1  2  3  4' 'CZ 1  4' 'CZ 2  4' 'H 1  2  3' 'Permute 1  4  2  3'
 'CNOT 1  2' 'Permute 2  1  4  3' 'CNOT 4  2' 'CNOT 2  4' 'CNOT 1  2'
 'CNOT 1  4' 'H 1  2  3  4' 'CZ 1  4' 'CZ 2  4' 'H 1  2  3'
 'Permute 1  4  2  3' 'CNOT 1  2']
Compiler is: 1
{4: 1, 3: 0, 2: 3, 1: 2}
{4

Got a 502 code response to https://quantumexperience.ng.bluemix.net/api/Jobs/5b560c885dc974003b7afe5a?access_token=EJGfoAxmiRZG2CkbpUx0bCZzqL3DJLkxW4FUAAqluGU87PXGfDuQCO362ismO6NB: 502 Bad Gateway: Registered endpoint failed to handle the request.



{'0000': 116, '0001': 43, '0010': 99, '0011': 47, '0100': 71, '0101': 42, '0110': 73, '0111': 46, '1000': 119, '1001': 24, '1010': 61, '1011': 42, '1100': 103, '1101': 34, '1110': 71, '1111': 33}
{4: 1, 3: 0, 2: 3, 1: 2}
{4: 0, 3: 3, 2: 1, 1: 2}
{4: 3, 3: 0, 2: 2, 1: 1}
{4: 0, 3: 2, 2: 3, 1: 1}


Got a 502 code response to https://quantumexperience.ng.bluemix.net/api/Jobs/5b560f1aa94755003ab99907?access_token=EJGfoAxmiRZG2CkbpUx0bCZzqL3DJLkxW4FUAAqluGU87PXGfDuQCO362ismO6NB: 502 Bad Gateway: Registered endpoint failed to handle the request.

Got a 502 code response to https://quantumexperience.ng.bluemix.net/api/Jobs/5b560f1aa94755003ab99907?access_token=EJGfoAxmiRZG2CkbpUx0bCZzqL3DJLkxW4FUAAqluGU87PXGfDuQCO362ismO6NB: 502 Bad Gateway: Registered endpoint failed to handle the request.



{'0000': 69, '0001': 81, '1000': 101, '1001': 46, '0100': 64, '0101': 60, '1100': 37, '1101': 48, '0010': 65, '0011': 42, '1010': 83, '1011': 42, '0110': 68, '0111': 99, '1110': 56, '1111': 63}
{4: 1, 3: 0, 2: 3, 1: 2}
{4: 0, 3: 3, 2: 1, 1: 2}
{4: 3, 3: 0, 2: 2, 1: 1}
{4: 0, 3: 2, 2: 3, 1: 1}
{'0000': 146, '0001': 73, '0010': 65, '0011': 34, '0100': 69, '0101': 35, '0110': 69, '0111': 23, '1000': 63, '1001': 54, '1010': 78, '1011': 43, '1100': 90, '1101': 42, '1110': 77, '1111': 63}
{4: 1, 3: 0, 2: 3, 1: 2}
{4: 0, 3: 3, 2: 1, 1: 2}
{4: 3, 3: 0, 2: 2, 1: 1}
{4: 0, 3: 2, 2: 3, 1: 1}


...

{'0000': 119, '0010': 95, '0001': 59, '0011': 40, '0100': 57, '0110': 61, '0101': 27, '0111': 26, '1000': 96, '1010': 93, '1001': 65, '1011': 26, '1100': 117, '1110': 77, '1101': 42, '1111': 24}
{4: 1, 3: 0, 2: 3, 1: 2}
{4: 0, 3: 3, 2: 1, 1: 2}
{4: 3, 3: 0, 2: 2, 1: 1}
{4: 0, 3: 2, 2: 3, 1: 1}


Got a 502 code response to https://quantumexperience.ng.bluemix.net/api/Jobs/5b562ca2f0e601003b96fa2c?access_token=EJGfoAxmiRZG2CkbpUx0bCZzqL3DJLkxW4FUAAqluGU87PXGfDuQCO362ismO6NB: 502 Bad Gateway: Registered endpoint failed to handle the request.



{'0000': 75, '1000': 81, '0001': 69, '1001': 65, '0100': 62, '1100': 34, '0101': 30, '1101': 27, '0010': 74, '1010': 76, '0011': 49, '1011': 96, '0110': 106, '1110': 53, '0111': 69, '1111': 58}
{4: 1, 3: 0, 2: 3, 1: 2}
{4: 0, 3: 3, 2: 1, 1: 2}
{4: 3, 3: 0, 2: 2, 1: 1}
{4: 0, 3: 2, 2: 3, 1: 1}


Got a 502 code response to https://quantumexperience.ng.bluemix.net/api/Jobs/5b562ec7f0e601003b96fa37?access_token=EJGfoAxmiRZG2CkbpUx0bCZzqL3DJLkxW4FUAAqluGU87PXGfDuQCO362ismO6NB: 502 Bad Gateway: Registered endpoint failed to handle the request.

Got a 502 code response to https://quantumexperience.ng.bluemix.net/api/Jobs/5b562ec7f0e601003b96fa37?access_token=EJGfoAxmiRZG2CkbpUx0bCZzqL3DJLkxW4FUAAqluGU87PXGfDuQCO362ismO6NB: 502 Bad Gateway: Registered endpoint failed to handle the request.



{'0000': 92, '0010': 81, '0001': 54, '0011': 29, '0100': 74, '0110': 83, '0101': 24, '0111': 20, '1000': 116, '1010': 77, '1001': 88, '1011': 35, '1100': 94, '1110': 106, '1101': 33, '1111': 18}
{4: 1, 3: 0, 2: 3, 1: 2}
{4: 0, 3: 3, 2: 1, 1: 2}
{4: 3, 3: 0, 2: 2, 1: 1}
{4: 0, 3: 2, 2: 3, 1: 1}
{'0000': 107, '1000': 122, '0001': 64, '1001': 61, '0100': 72, '1100': 64, '0101': 19, '1101': 20, '0010': 80, '1010': 93, '0011': 33, '1011': 33, '0110': 131, '1110': 83, '0111': 27, '1111': 15}
{4: 1, 3: 0, 2: 3, 1: 2}
{4: 0, 3: 3, 2: 1, 1: 2}
{4: 3, 3: 0, 2: 2, 1: 1}
{4: 0, 3: 2, 2: 3, 1: 1}
{'0000': 127, '0010': 93, '0001': 53, '0011': 27, '1000': 70, '1010': 50, '1001': 30, '1011': 24, '0100': 117, '0110': 81, '0101': 44, '0111': 34, '1100': 97, '1110': 104, '1101': 48, '1111': 25}
{4: 1, 3: 0, 2: 3, 1: 2}
{4: 0, 3: 3, 2: 1, 1: 2}
{4: 3, 3: 0, 2: 2, 1: 1}
{4: 0, 3: 2, 2: 3, 1: 1}
{'0000': 95, '0010': 106, '0001': 63, '0011': 44, '0100': 69, '0110': 46, '0101': 19, '0111': 33, '1000': 91, '10